# Scikit-learn Beginner's Template

---

## Selected Machine Learning References

### Scikit-learn

* Documentation: http://scikit-learn.org/stable/documentation.html
* User guide: http://scikit-learn.org/stable/user_guide.html

### ML and Python

* Python Data Science Handbook, Jake VanderPlas: https://github.com/jakevdp/PythonDataScienceHandbook

### ML in general

* List of HEP-ML resources: https://github.com/iml-wg/HEP-ML-Resources
* Stanford lecture series on Machine Learning by Andrej Karpathy: https://www.youtube.com/playlist?list=PLkt2uSq6rBVctENoVBg1TpCC7OQi31AlC

---

## Example Analysis

### Preparatory Steps

_python imports, random seed fixing_

### Data Import

### Feature Engineering ("Defining X")

### Target Vector Definition ("Defining y")

### Data Preprocessing

_feature scaling, (PCA), ..._

### Data Split Into Training, Validation and Test Sample

### Model Definition

### Model Training

_model training, variable importances, ..._

---

### Model Evaluation on the Validation Sample

#### MVA output distribution

#### Cut efficiencies plot / MVA cut optimization

#### ROC curve

#### Precision-recall curve

#### Confusion matrix

#### Classification report

---

### Model Application to the Test Sample

#### MVA output distribution

#### ROC curve

#### Precision-recall curve

#### Confusion matrix

#### Classification report